# Parse Documents Xml to Json

In [1]:
import xmltodict, json, os
from tqdm import tqdm

In [2]:
def parseDocXml2Json(doc):
  docJson = {}
  
  with open(doc, 'r') as file:
    obj = xmltodict.parse(file.read())

  for field in obj['add']['doc']['field']:
    name = field['@name']
    content = field['#text']
    docJson[name] = content
  return docJson

# Elastic Search First Steps

In [14]:
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch("http://localhost:9200")

In [15]:
doc = {
    'author': 'kimchy',
    'text': 'Elasticsearch: cool. bonsai cool.',
    'timestamp': datetime.now(),
}

In [17]:
resp = es.index(index="test-index", id=1, document=doc)
print(resp['result'])

ConnectionTimeout: Connection timed out

In [ ]:

resp = es.get(index="test-index", id=1)
print(resp['_source'])

In [ ]:
es.indices.refresh(index="test-index")

In [ ]:
resp = es.search(index="test-index", query={"match_all": {}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

In [ ]:
hit

# First try 

In [5]:
docJson = parseDocXml2Json('./documents/BR-BG.03780.xml')
docJson

{'docid': 'BR-BG.03780',
 'filename': 'BGP_2013_21_2_14_Capa_3.pdf',
 'filetype': 'PDF',
 'text': 'contornitos e as correntes de Fundo\nContornitos, por definição, são "sedimentos depositados ou intensamente retrabalhados pela ação persistente de correntes de fundo". Na Petrobras, sua identificação e estudo tiveram início na década de 1980 em depósitos eocênicos na Bacia de Campos. Diversos trabalhos internos e acadêmicos nos anos seguintes foram dedicados à compreensão dos processos de re-trabalhamento de depósitos turbidíticos.\nCorrentes de fundo se originam pela movimentação relativa de corpos de água determinada por diferenças de densidade, em função de variação na temperatura e salinidade. A velocidade destas correntes seria correlacionável com a intensidade do gradiente de densidade. Há registros de correntes que alcançam velocidade de até mesmo 3ms-1, como no Golfo de Cádiz (Espanha). A interação destas correntes com a fisiografia do fundo também pode aumentar ou diminuir sua v

In [6]:
text = docJson['text'].lower()
slicer = text.find('\n')
title, content = text[:slicer], text[slicer+1:]
doc = {
  'id': docJson['docid'],
  'title': title,
  'content': content,
  'timestamp': datetime.now(),
}
doc

{'id': 'BR-BG.03780',
 'title': 'contornitos e as correntes de fundo',
 'content': 'contornitos, por definição, são "sedimentos depositados ou intensamente retrabalhados pela ação persistente de correntes de fundo". na petrobras, sua identificação e estudo tiveram início na década de 1980 em depósitos eocênicos na bacia de campos. diversos trabalhos internos e acadêmicos nos anos seguintes foram dedicados à compreensão dos processos de re-trabalhamento de depósitos turbidíticos.\ncorrentes de fundo se originam pela movimentação relativa de corpos de água determinada por diferenças de densidade, em função de variação na temperatura e salinidade. a velocidade destas correntes seria correlacionável com a intensidade do gradiente de densidade. há registros de correntes que alcançam velocidade de até mesmo 3ms-1, como no golfo de cádiz (espanha). a interação destas correntes com a fisiografia do fundo também pode aumentar ou diminuir sua velocidade. desta forma, as correntes são muito afeta

In [7]:
resp = es.index(index="test-petrobras", id=1, document=doc)
print(resp['result'])

ConnectionTimeout: Connection timed out

In [8]:
resp = es.get(index="test-petrobras", id=1)
print(resp['_source'])

{'id': 'BR-BG.03780', 'title': 'contornitos e as correntes de fundo', 'content': 'contornitos, por definição, são "sedimentos depositados ou intensamente retrabalhados pela ação persistente de correntes de fundo". na petrobras, sua identificação e estudo tiveram início na década de 1980 em depósitos eocênicos na bacia de campos. diversos trabalhos internos e acadêmicos nos anos seguintes foram dedicados à compreensão dos processos de re-trabalhamento de depósitos turbidíticos.\ncorrentes de fundo se originam pela movimentação relativa de corpos de água determinada por diferenças de densidade, em função de variação na temperatura e salinidade. a velocidade destas correntes seria correlacionável com a intensidade do gradiente de densidade. há registros de correntes que alcançam velocidade de até mesmo 3ms-1, como no golfo de cádiz (espanha). a interação destas correntes com a fisiografia do fundo também pode aumentar ou diminuir sua velocidade. desta forma, as correntes são muito afetada

In [9]:
es.indices.refresh(index="test-petrobras")

ObjectApiResponse({'_shards': {'total': 2, 'successful': 1, 'failed': 0}})

In [17]:
resp = es.search(index="test-petrobras", query={"match": { "content": "carro de bebida"}})
print("Got %d Hits:" % resp['hits']['total']['value'])
for hit in resp['hits']['hits']:
    timestamp, title, content, score = hit["_source"]['timestamp'], hit["_source"]['title'], hit["_source"]['content'], hit['_score']
    print(f"{timestamp} ({score}) {title}: {content[:50]}...")

Got 9857 Hits:
2023-01-24T22:02:28.626879 (10.928932) universidade federal de santa catarina: centro sócio-económico

departamento de serviço so...
2023-01-24T22:00:03.584082 (10.399542) juliana feliciano de almeida :  
 
 
 
 
 
 
 
 
 
 
 
 
 

exposição à violência...
2023-01-24T21:58:40.933339 (10.1411495) universidade de brasília :        instituto de psicologia 

 

 

 

 

 

 
...
2023-01-24T22:01:29.280942 (9.971802) i : 
 
 

 

universidade de aveiro 

2016  

departam...
2023-01-24T22:00:25.166311 (9.957164) josé marcos sinhorini : 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

a co...
2023-01-24T21:58:37.696219 (9.921247) 1 :  

 

 

 

 

 

centro universitário univates 

...
2023-01-24T22:01:55.336925 (9.622571) universidade estadual de campinas : 
instituto de artes 

 
 

 

 

 

 

 

 

 

 
...
2023-01-24T22:01:04.730124 (9.189025) universidade estadual de campinas faculdade de ciências médicas departamento de medicina preventiva e social: transitando entre o col

In [18]:
resp['hits']['hits'][0]

{'_index': 'test-petrobras',
 '_id': '7473',
 '_score': 10.928932,
 '_ignored': ['content.keyword'],
 '_source': {'id': 'BR-TU.14675',
  'title': 'universidade federal de santa catarina',
  'content': 'centro sócio-económico\n\ndepartamento de serviço social\n\nautora: alice dolor pereira\n\npossibilidades e limites do serviço social na casan/arfl:\n\natendimento aos familiares de funcionários com problemas\n\nrelacionados ao abuso de drogas.\n\nflorianópolis - sc\n2004\n\n\n\nalice dolor pereira\n\npossibilidades e limites do serviço social na casan/arfl:\n\natendimento aos familiares de funcionários com problemas\n\nrelacionados ao abuso de drogas.\n\ntrabalho de conclusão de curso apresentado ao\ndepartamento de serviço social da universidade\nfederal de santa catarina. para obtenção do titulo\nde bacharel em serviço social, sob orientação da\nprofessora jaira freixiela adamczyk.\n\n,//\nter -gk á\t oa\nchefe o de o.\t serviço social\n\n,cse/ufsc\n\ndepto. serviço social\n\ndefendid

In [4]:
parseDocuments = []
documents = os.listdir('./documents/')
id = 1
for doc in tqdm(documents):
  docJson = parseDocXml2Json(f'./documents/{doc}')

  text = docJson['text'].lower()
  slicer = text.find('\n')
  title, content = text[:slicer], text[slicer+1:]
  
  doc = {
    'id': docJson['docid'],
    'title': title,
    'content': content,
    'timestamp': datetime.now(),
  }

  resp = es.index(index="test-petrobras", id=id, document=doc)

  if(id%100):
    es.indices.refresh(index="test-petrobras")

  id+=1

  0%|          | 0/21444 [00:10<?, ?it/s]


ConnectionTimeout: Connection timed out